# Imports

In [177]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

In [1]:
import pandas as pd
import numpy as np

In [6]:
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
pio.templates.default = "plotly"

In [50]:
from numba import njit

# Data Set Information

The dataset contains **9358** instances of hourly averaged responses from an array of **5 metal oxide chemical** sensors embedded in an **Air Quality Chemical Multisensor Device**. The device was located on the field in a significantly polluted area, at road level,within an Italian city. Data were recorded from **March 2004 to February 2005** (one year)representing the longest freely available recordings of on field deployed air quality chemical sensor devices responses. Ground Truth hourly averaged concentrations for CO, Non Metanic Hydrocarbons, Benzene, Total Nitrogen Oxides (NOx) and Nitrogen Dioxide (NO2) and were provided by a co-located reference certified analyzer. Evidences of cross-sensitivities as well as both concept and sensor drifts are present as described in De Vito et al., Sens. And Act. B, Vol. 129,2,2008 (citation required) eventually affecting sensors concentration estimation capabilities. **Missing values are tagged with -200 value.**

In [108]:
data = pd.read_csv('AirQualityUCI.csv',sep=';')

# Attribute Information

0- **Date** (DD/MM/YYYY) <br>
1- **Time** (HH.MM.SS)<br>
2- True hourly averaged concentration **CO in mg/m^3** (reference analyzer)<br>
3- **PT08.S1** (tin oxide) hourly averaged sensor response (nominally CO targeted)
4- True hourly averaged overall **Non Metanic HydroCarbons concentration in microg/m^3** (reference analyzer)<br>
5- True hourly averaged** Benzene concentration in microg/m^3** (reference analyzer)<br>
6- **PT08.S2** (titania) hourly averaged sensor response (nominally NMHC targeted)<br>
7- True hourly averaged **NOx concentration in ppb** (reference analyzer)<br>
8- **PT08.S3** (tungsten oxide) hourly averaged sensor response (nominally NOx targeted)<br>
9- True hourly averaged **NO2 concentration in microg/m^3** (reference analyzer)<br>
10- **PT08.S4**(tungsten oxide) hourly averaged sensor response (nominally NO2 targeted)<br>
11- **PT08.S5** (indium oxide) hourly averaged sensor response (nominally O3 targeted)<br>
12- **Temperature** in °C<br>
13- **Relative Humidity** (%)<br>
14- **AH** Absolute Humidity<br>

In [15]:
data.head()

,Date,Time,CO(GT),PT08.S1(CO),NMHC(GT),C6H6(GT),PT08.S2(NMHC),NOx(GT),PT08.S3(NOx),NO2(GT),PT08.S4(NO2),PT08.S5(O3),T,RH,AH,Unnamed: 15,Unnamed: 16
0,10/03/2004,18.00.00,"2,6",1360.0,150.0,"11,9",1046.0,166.0,1056.0,113.0,1692.0,1268.0,"13,6","48,9","0,7578",NaN,NaN
1,10/03/2004,19.00.00,2,1292.0,112.0,"9,4",955.0,103.0,1174.0,92.0,1559.0,972.0,"13,3","47,7","0,7255",NaN,NaN
2,10/03/2004,20.00.00,"2,2",1402.0,88.0,"9,0",939.0,131.0,1140.0,114.0,1555.0,1074.0,"11,9","54,0","0,7502",NaN,NaN
3,10/03/2004,21.00.00,"2,2",1376.0,80.0,"9,2",948.0,172.0,1092.0,122.0,1584.0,1203.0,"11,0","60,0","0,7867",NaN,NaN
4,10/03/2004,22.00.00,"1,6",1272.0,51.0,"6,5",836.0,131.0,1205.0,116.0,1490.0,1110.0,"11,2","59,6","0,7888",NaN,NaN


# Anomaly detection

For **simplicity reasons**, let's pick a specific feature and in a specific time and try to observe and detect outliers. In real life cases, detecting outliers is much more complicated since contextual variables must be included in the equation.**Contextual variables** can be seasons, trends or whatever variable that generates what it seems to be an outlier but it isn't.

In [178]:
time = '00.00.00'
feature = 'NOx(GT)'
px.line(x=data.loc[data.Time == time, 'Date'], y=data.loc[data.Time == time, feature])

In [213]:
def compute_iqr_v1(data, feature='NOx(GT)', coef=1.5):
    @njit
    def compute_low_up(q3,q1):
        iqr = q3 - q1
        low = q1 - coef * iqr
        up = q3 + coef * iqr
        return low, up
    # It does ot make sens to compute quantiles with noisy data points
    q3 = data.loc[(~((data[feature] == -200)&(data[feature].isna()))) , feature].quantile(.75)
    q1 = data.loc[(~((data[feature] == -200)&(data[feature].isna()))) , feature].quantile(.25)
    low, up = compute_low_up(q3,q1)
    data.insert(len(data.columns), 'IQR_OUTLIER', ((data[feature] < low) | (data[feature] > up)).astype(int))
    iqr_outliers = ((data[feature] < low) | (data[feature] > up)).astype(int)

    return iqr_outliers, low, up

In [350]:
df = data.loc[data.Time == time]
outliers, low, up = compute_iqr_v1(df, feature='NOx(GT)')

In [153]:
def setcolor(x):
    if(x == 1):
        return "FireBrick"
    else:
        return "#00CC96"

In [351]:
fig = go.Figure()
fig.add_hline(y=low, line_width=1.5, line_dash="dash", line_color="black")
fig.add_hrect(y0=low, y1=df[feature].min()-100, line_width=0, fillcolor="crimson", opacity=0.2, annotation_text="OUTLIERS")
fig.add_hrect(y0=low, y1=up, line_width=0, fillcolor="#00CC96", opacity=0.2, annotation_text="INLIERS")

fig.add_trace(go.Scatter(x=df['Date'], 
                         y=df[feature],
                         marker = dict(color=list(map(setcolor, outliers))),
                         mode="markers"))
fig.add_hline(y=up, line_width=1.5, line_dash="dash", line_color="black")
fig.add_hrect(y0=up, y1=df[feature].max()+100, line_width=0, fillcolor="crimson", opacity=0.2, annotation_text="OUTLIERS")

fig.update_layout(
    title=feature+" values where time = "+time,
    xaxis_title="Date", 
    yaxis_title='Average NOx concentration in ppb '
)
fig.show()

Using the IQR we were able to detect the outliers however, we're computing the IQR value on the whole vector. This means that we're not taking into consideration the possible evolution of our observed feature (i.e. NOx). Therefore, computing IQR on shorter periods provides much more flexibility. 

In [323]:
def compute_iqr_v2(data, feature='NOx(GT)', step=30,  coef=1.5):
    
        initital_index = 0
        iqr_window = step
        iqr_outliers = np.array([])
        lower_bounds = np.array([])
        upper_bounds = np.array([])
        residual_flag = True
        length = len(data[feature])
        while iqr_window <= length:

            local_vector = data.loc[(~((data[feature] == -200)&(data[feature].isna()))) , feature].to_numpy()[initital_index:iqr_window]

            q3 = np.quantile(local_vector, .75)
            q1 = np.quantile(local_vector, .25)
            iqr = q3 - q1
            low = q1 - coef * iqr
            up = q3 + coef * iqr

            window_outliers = ((local_vector < low) | (local_vector > up)).astype(int)
            iqr_outliers = np.append(iqr_outliers, window_outliers)
            lower_bounds = np.append(lower_bounds, low)
            upper_bounds = np.append(upper_bounds, up)

            initital_index = iqr_window
            iqr_window += step
            if iqr_window == length:  # the split provides equal chunks
                residual_flag = False

        if residual_flag:
            local_vector = data.loc[(~((data[feature] == -200)&(data[feature].isna()))), feature].to_numpy()[iqr_window-step:length]
            q3 = np.quantile(local_vector, .75)
            q1 = np.quantile(local_vector, .25)
            iqr = q3 - q1
            low = q1 - coef * iqr
            up = q3 + coef * iqr
            window_outliers = ((local_vector < low) | (
                local_vector > up)).astype(int)
            iqr_outliers = np.append(iqr_outliers, window_outliers)
            lower_bounds = np.append(lower_bounds, low)
            upper_bounds = np.append(upper_bounds, up)
            
        plot_lower_bounds = lower_bounds.copy()
        plot_upper_bounds = upper_bounds.copy()
        lower_bounds = np.repeat(lower_bounds, step)
        upper_bounds = np.repeat(upper_bounds, step)
        return iqr_outliers, lower_bounds, upper_bounds, plot_lower_bounds, plot_upper_bounds

In [334]:
step=30
coef=1
iqr_outliers, lower_bounds, upper_bounds, plot_lower_bounds, plot_upper_bounds  = compute_iqr_v2(df, step=step,feature='NOx(GT)',coef=coef)

In [376]:
fig = go.Figure()

x0,x1=0,step
for low,up in zip(plot_lower_bounds,plot_upper_bounds):
    
    fig.add_shape(type="rect", x0=x0, y0=low, x1=x1, y1=plot_lower_bounds.min()-100,
    line=dict(width=0), fillcolor="crimson", opacity=0.2)
    
    fig.add_shape(type="rect", x0=x0, y0=up, x1=x1, y1=df[feature].max()+100,
    line=dict(width=0), fillcolor="crimson", opacity=0.2)

    x0+=step
    x1+=step
    
fig.add_trace(go.Scatter(x=df['Date'], 
                         y=upper_bounds,
                         line = dict(color='black', dash='dash'),
                         mode="lines", name='Upper Bound' ))


fig.add_trace(go.Scatter(x=df['Date'], 
                         y=lower_bounds,
                         line = dict(color='black',dash='dash'),
                         mode="lines", name='Lower Bound', fill='tonextx'  ,fillcolor='rgba(0,255,188,0.2)'))



fig.add_trace(go.Scatter(x=df['Date'], 
                         y=df[feature],
                         marker = dict(color=list(map(setcolor, iqr_outliers))),
                         mode="markers", name=' NOx concentration'))

fig.update_layout(
    title=feature+" values where time = "+time+" | IQR window = "+str(step),
    xaxis_title="Date", 
    yaxis_title='Average NOx concentration in ppb '
)
fig.show()

In [388]:
step=60
coef=1
iqr_outliers, lower_bounds, upper_bounds, plot_lower_bounds, plot_upper_bounds  = compute_iqr_v2(df, step=step,feature='NOx(GT)',coef=coef)

In [390]:
fig = go.Figure()

x0,x1=0,step
for low,up in zip(plot_lower_bounds,plot_upper_bounds):
    
    fig.add_shape(type="rect", x0=x0, y0=low, x1=x1, y1=plot_lower_bounds.min()-100,
    line=dict(width=0), fillcolor="crimson", opacity=0.2)
    
    fig.add_shape(type="rect", x0=x0, y0=up, x1=x1, y1=df[feature].max()+100,
    line=dict(width=0), fillcolor="crimson", opacity=0.2)

    x0+=step
    x1+=step
    
fig.add_trace(go.Scatter(x=df['Date'], 
                         y=upper_bounds,
                         line = dict(color='black', dash='dash'),
                         mode="lines", name='Upper Bound' ))


fig.add_trace(go.Scatter(x=df['Date'], 
                         y=lower_bounds,
                         line = dict(color='black',dash='dash'),
                         mode="lines", name='Lower Bound', fill='tonextx'  ,fillcolor='rgba(0,255,188,0.2)'))



fig.add_trace(go.Scatter(x=df['Date'], 
                         y=df[feature],
                         marker = dict(color=list(map(setcolor, iqr_outliers))),
                         mode="markers", name=' NOx concentration'))

fig.update_layout(
    title=feature+" values where time = "+time+" | IQR window = "+str(step),
    xaxis_title="Date", 
    yaxis_title='Average NOx concentration in ppb '
)
fig.show()

In [395]:
step=90
coef=1
iqr_outliers, lower_bounds, upper_bounds, plot_lower_bounds, plot_upper_bounds  = compute_iqr_v2(df, step=step,feature='NOx(GT)',coef=coef)

In [396]:
fig = go.Figure()

x0,x1=0,step
for low,up in zip(plot_lower_bounds,plot_upper_bounds):
    
    fig.add_shape(type="rect", x0=x0, y0=low, x1=x1, y1=plot_lower_bounds.min()-100,
    line=dict(width=0), fillcolor="crimson", opacity=0.2)
    
    fig.add_shape(type="rect", x0=x0, y0=up, x1=x1, y1=df[feature].max()+100,
    line=dict(width=0), fillcolor="crimson", opacity=0.2)

    x0+=step
    x1+=step
    
fig.add_trace(go.Scatter(x=df['Date'], 
                         y=upper_bounds,
                         line = dict(color='black', dash='dash'),
                         mode="lines", name='Upper Bound' ))


fig.add_trace(go.Scatter(x=df['Date'], 
                         y=lower_bounds,
                         line = dict(color='black',dash='dash'),
                         mode="lines", name='Lower Bound', fill='tonextx'  ,fillcolor='rgba(0,255,188,0.2)'))



fig.add_trace(go.Scatter(x=df['Date'], 
                         y=df[feature],
                         marker = dict(color=list(map(setcolor, iqr_outliers))),
                         mode="markers", name=' NOx concentration'))

fig.update_layout(
    title=feature+" values where time = "+time+" | IQR window = "+str(step),
    xaxis_title="Date", 
    yaxis_title='Average NOx concentration in ppb '
)
fig.show()